In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import numpy as np

from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Industrial_and_Scientific_5.json.gz
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Luxury_Beauty_5.json.gz

--2021-06-05 14:29:38--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Industrial_and_Scientific_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10994038 (10M) [application/octet-stream]
Saving to: ‘Industrial_and_Scientific_5.json.gz’

Industrial_and_Scie 100%[===================>]  10.48M  30.5MB/s    in 0.3s    

2021-06-05 14:29:38 (30.5 MB/s) - ‘Industrial_and_Scientific_5.json.gz’ saved [10994038/10994038]

--2021-06-05 14:29:38--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Luxury_Beauty_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7356868 (7.0M) [application/octet-stream]
Saving to: ‘Luxury_Beauty_5.json.gz’

Luxury_Beauty_

In [ ]:
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import en_core_web_sm

# Load the spacy model that you have installed
nlp = en_core_web_sm.load()

# process a sentence using the model
doc = nlp("This is some text that I am processing with Spacy")

# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'text':
doc[3].vector

# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector

array([-1.5933637e-01,  1.0727510e+00,  5.8144492e-01, -5.5949199e-01,
        5.8384252e-01,  6.2737018e-01,  2.0882721e+00,  5.3056949e-01,
        1.2801822e+00,  1.0998030e+00, -4.0343684e-01,  4.1303840e-01,
       -8.6142081e-01,  1.3498663e+00,  7.1843758e-02, -2.6160771e-01,
       -1.2563832e+00, -2.2701338e-01, -3.3985606e-01, -4.9475414e-01,
        1.0330565e+00,  4.9247903e-01, -1.0581312e-01, -4.4477254e-01,
       -1.7779062e+00,  3.8878676e-01, -1.6339018e+00, -6.1836970e-01,
        2.5130153e-01, -5.3813636e-01,  1.9087858e+00,  1.1364286e+00,
       -4.3658358e-01, -6.2360942e-01,  3.6663722e-02, -1.1008348e+00,
        4.4784656e-01, -3.6548299e-01, -1.0427644e+00, -2.4165368e-01,
        8.7189072e-01, -6.1751854e-01,  1.0153117e+00, -1.4855057e+00,
        1.1795838e+00, -7.4856341e-02, -7.4507809e-01, -1.9475173e-02,
       -1.8662630e+00,  1.5052016e+00,  1.5552965e+00, -1.0483619e+00,
        4.1499394e-01,  7.5322583e-02, -7.8286880e-01,  8.3658588e-01,
      

In [ ]:
doc[3].vector.shape

(96,)

In [ ]:
### load the meta data

data = []
with gzip.open('Industrial_and_Scientific_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

77071
{'overall': 5.0, 'verified': True, 'reviewTime': '11 27, 2017', 'reviewerID': 'A1JB7HFWHRYHT7', 'asin': 'B0000223SI', 'style': {'Size:': ' 1-(Pack)'}, 'reviewerName': 'Alex W.', 'reviewText': "This worked really well for what I used it for. So for my purposes it is getting full marks. This is an all around great, durable, and afforable sandpaper.\n\nPros:\n-Grit cuts really fast and evenly. No random deep scratches like I have seen in some cheaper paper\n-Didn't even have a hint of clogging up.\n-The adhesive is just what I needed. No permanent, but wasn't going anywhere.\n\nCons:\n-None", 'summary': "Couldn't have been happier with it's performance", 'unixReviewTime': 1511740800}


In [ ]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)
del data
print(len(df))

77071


In [ ]:
### remove rows with unformatted title (i.e. some 'title' may still contain html style content)

data = df.fillna('')[["reviewText", "overall"]]
del df

In [ ]:
res = data[:34000].progress_apply(lambda x: np.hstack([nlp(x['reviewText']).vector, x['overall']]),  axis=1)

In [ ]:
np.save("Industrial_and_Scientific_5_emb", np.vstack([i for i in res.to_list() if i.size == 97]))

In [ ]:
np.vstack([i for i in res.to_list() if i.size == 97]).shape

(33996, 97)